In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [14]:
# read in CSV
hmb_1_imu = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/imu-data.csv')
hmb_1_imu.isnull().sum()

Time                                0
header.seq                          0
header.stamp.secs                   0
header.stamp.nsecs                  0
header.frame_id                     0
orientation.x                       0
orientation.y                       0
orientation.z                       0
orientation.w                       0
orientation_covariance_0            0
orientation_covariance_1            0
orientation_covariance_2            0
orientation_covariance_3            0
orientation_covariance_4            0
orientation_covariance_5            0
orientation_covariance_6            0
orientation_covariance_7            0
orientation_covariance_8            0
angular_velocity.x                  0
angular_velocity.y                  0
angular_velocity.z                  0
angular_velocity_covariance_0       0
angular_velocity_covariance_1       0
angular_velocity_covariance_2       0
angular_velocity_covariance_3       0
angular_velocity_covariance_4       0
angular_velo

In [4]:
fix_HMB_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/fix.csv')
fix_HMB_1['position_covariance_type'].unique()

array([0], dtype=int64)

In [12]:
fix_HMB_1 = pd.read_csv(f'./Data/ROSBAG/Ch2_002/HMB_1/fix.csv')
def clean_fix_csv(df,filename):
    df.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs', 'header.frame_id','status.status','status.service','position_covariance_type',
                       'position_covariance_0','position_covariance_1','position_covariance_2','position_covariance_3',
                      'position_covariance_4','position_covariance_5','position_covariance_6','position_covariance_7','position_covariance_8'], inplace = True)
    df.to_csv(f'./cleaned_data/{filename}.csv')
    return df
    
clean_fix_csv(fix_HMB_1,'fix_HMB_1')

,Time,latitude,longitude,altitude
0,1.479424e+09,37.545429,-122.326279,6.665314
1,1.479424e+09,37.545429,-122.326279,6.668632
2,1.479424e+09,37.545425,-122.326279,6.677958
3,1.479424e+09,37.545425,-122.326279,6.681274
4,1.479424e+09,37.545425,-122.326286,6.684596
...,...,...,...,...
88468,1.479424e+09,37.507816,-122.343414,84.744789
88469,1.479424e+09,37.507816,-122.343414,84.746536
88470,1.479424e+09,37.507816,-122.343414,84.748314
88471,1.479424e+09,37.507816,-122.343414,84.750099


In [15]:
def clean_imu_csv(df,filename):
    df.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs', 'header.frame_id'])
    df.to_csv(f'./cleaned_data/{filename}.csv')
    return df
clean_imu_csv(hmb_1_imu,'imu_HMB_1')

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,...,linear_acceleration.z,linear_acceleration_covariance_0,linear_acceleration_covariance_1,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8
0,1.479424e+09,1183922,1479424214,778361082,/imu,0.012995,0.028516,0.919373,0.392138,0.017453,...,8.583070,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
1,1.479424e+09,1183923,1479424214,780781030,/imu,0.012959,0.028632,0.919347,0.392191,0.017453,...,9.315427,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
2,1.479424e+09,1183924,1479424214,783006906,/imu,0.012934,0.028753,0.919331,0.392220,0.017453,...,10.403072,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
3,1.479424e+09,1183925,1479424214,784673929,/imu,0.012914,0.028845,0.919311,0.392260,0.017453,...,10.644299,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
4,1.479424e+09,1183926,1479424214,785916090,/imu,0.012933,0.028897,0.919291,0.392304,0.017453,...,11.602013,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88467,1.479424e+09,1272389,1479424435,943310022,/imu,0.020810,0.026328,0.932314,-0.360091,0.017453,...,12.772095,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
88468,1.479424e+09,1272390,1479424435,945827007,/imu,0.020748,0.026231,0.932281,-0.360186,0.017453,...,12.600610,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
88469,1.479424e+09,1272391,1479424435,948255062,/imu,0.020697,0.026121,0.932239,-0.360306,0.017453,...,12.607566,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004
88470,1.479424e+09,1272392,1479424435,950861930,/imu,0.020663,0.025985,0.932200,-0.360418,0.017453,...,12.645799,0.0004,0.0,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004


In [17]:
brake_HMB_1 = pd.read_csv(f'./Data/ROSBAG/Ch2_002/HMB_1/vehicle-brake_info_report.csv')
def clean_brake_csv(df,filename):
    df['stationary'] = df['stationary'].astype(int) 
    df = df[['Time','brake_torque_actual','wheel_torque_actual','accel_over_ground','stationary']]
    df.to_csv(f'./cleaned_data/{filename}.csv')
    return df
clean_brake_csv(brake_HMB_1,'brake_HMB_1')

,Time,brake_torque_actual,wheel_torque_actual,accel_over_ground,stationary
0,1.479424e+09,0.0,620.0,0.070,0
1,1.479424e+09,0.0,608.0,0.070,0
2,1.479424e+09,0.0,612.0,0.105,0
3,1.479424e+09,0.0,616.0,0.105,0
4,1.479424e+09,0.0,620.0,0.105,0
...,...,...,...,...,...
11054,1.479424e+09,0.0,172.0,-0.280,0
11055,1.479424e+09,0.0,184.0,-0.280,0
11056,1.479424e+09,0.0,184.0,-0.315,0
11057,1.479424e+09,0.0,200.0,-0.315,0


In [3]:
def attach_image(dataframe,images):

    # Attatching image files to the inputs
    
    image_base_dir = f'./Data/ROSBAG/Ch2_002/images/{images}'
    
    # Define camera folders
    camera_folders = {
        "left": os.path.join(image_base_dir, "left"),
        "center": os.path.join(image_base_dir, "center"),
        "right": os.path.join(image_base_dir, "right"),
    }
    
    # Add new columns for image paths based on matching Time and frame filename
    for camera, folder in camera_folders.items():
        # Loop through each row and check for matching Time (without decimals) and image filename
        dataframe[f"{camera}_image_path"] = dataframe['Time'].apply(
            lambda x: os.path.join(folder, f'frame_{str(int(x))}.png').replace("\\", "/") if os.path.exists(os.path.join(folder, f'frame_{str(int(x))}.png')) else None
        )
    return dataframe
imu_image = attach_image(hmb_1_imu,'HMB_1')
imu_image.head()    

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,...,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8,left_image_path,center_image_path,right_image_path
0,1.479424e+09,1183922,1479424214,778361082,/imu,0.012995,0.028516,0.919373,0.392138,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,None,None,None
1,1.479424e+09,1183923,1479424214,780781030,/imu,0.012959,0.028632,0.919347,0.392191,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,None,None,None
2,1.479424e+09,1183924,1479424214,783006906,/imu,0.012934,0.028753,0.919331,0.392220,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,None,None,None
3,1.479424e+09,1183925,1479424214,784673929,/imu,0.012914,0.028845,0.919311,0.392260,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,None,None,None
4,1.479424e+09,1183926,1479424214,785916090,/imu,0.012933,0.028897,0.919291,0.392304,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,None,None,None


In [4]:
steer = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-steering_report.csv')
throttle = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-throttle_info_report.csv')

In [5]:
steer.shape

(11040, 17)

In [6]:
steer.isnull().sum()

Time                            0
header.seq                      0
header.stamp.secs               0
header.stamp.nsecs              0
header.frame_id             11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
dtype: int64

In [7]:
throttle.shape

(22118, 8)

In [8]:
throttle.isnull().sum()

Time                      0
header.seq                0
header.stamp.secs         0
header.stamp.nsecs        0
header.frame_id       22118
throttle_pc               0
throttle_rate             0
engine_rpm                0
dtype: int64

In [9]:
throttle_steer = steer.merge(throttle,on = 'Time',how = 'left')
throttle_steer.head()

,Time,header.seq_x,header.stamp.secs_x,header.stamp.nsecs_x,header.frame_id_x,steering_wheel_angle,steering_wheel_angle_cmd,steering_wheel_torque,speed,enabled,...,fault_bus2,fault_calibration,fault_connector,header.seq_y,header.stamp.secs_y,header.stamp.nsecs_y,header.frame_id_y,throttle_pc,throttle_rate,engine_rpm
0,1.479424e+09,278688,1479424214,787219361,NaN,0.003491,0.0,0.1250,22.880556,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.479424e+09,278689,1479424214,807231859,NaN,0.003491,0.0,0.1250,22.886110,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.479424e+09,278690,1479424214,827300730,NaN,0.003491,0.0,0.1250,22.891666,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.479424e+09,278691,1479424214,847355643,NaN,0.003491,0.0,0.1875,22.897223,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.479424e+09,278692,1479424214,867355824,NaN,0.003491,0.0,0.0000,22.902779,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
throttle_steer.isnull().sum()

Time                            0
header.seq_x                    0
header.stamp.secs_x             0
header.stamp.nsecs_x            0
header.frame_id_x           11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
header.seq_y                11040
header.stamp.secs_y         11040
header.stamp.nsecs_y        11040
header.frame_id_y           11040
throttle_pc                 11040
throttle_rate               11040
engine_rpm                  11040
dtype: int64

### Merging both the throttle and steer dataframes shows that there are no matching time frames for both datasets 

In [11]:
steer = attach_image(steer,'HMB_1')
throttle = attach_image(throttle,'HMB_1')

In [12]:
steer = steer.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id','steering_wheel_angle_cmd',
                              'enabled','override','driver','fault_wdc','fault_bus1','fault_bus2','fault_calibration','fault_connector'])
throttle = throttle.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id',])

# speed is in m/s so change to km/h
steer['speed'] = steer['speed'] * 3.6

steer.to_csv('./cleaned_data/steer_data_HMB_1.csv', index = False)
throttle.to_csv('./cleaned_data/throttle_data_HMB_1.csv', index = False)


In [15]:
steer.head()

,Time,steering_wheel_angle,steering_wheel_torque,speed,left_image_path,center_image_path,right_image_path
0,1.479424e+09,0.003491,0.1250,82.370002,None,None,None
1,1.479424e+09,0.003491,0.1250,82.389997,None,None,None
2,1.479424e+09,0.003491,0.1250,82.409999,None,None,None
3,1.479424e+09,0.003491,0.1875,82.430001,None,None,None
4,1.479424e+09,0.003491,0.0000,82.450003,None,None,None


In [18]:
steer['Time'].diff().describe()

count    11039.000000
mean         0.020034
std          0.000508
min          0.013540
25%          0.019989
50%          0.020033
75%          0.020078
max          0.026501
Name: Time, dtype: float64

In [19]:
throttle.head()

,Time,throttle_pc,throttle_rate,engine_rpm,left_image_path,center_image_path,right_image_path
0,1.479424e+09,0.418,0.0,2770.00,None,None,None
1,1.479424e+09,0.418,0.0,2762.25,None,None,None
2,1.479424e+09,0.418,0.0,2762.25,None,None,None
3,1.479424e+09,0.418,0.0,2754.75,None,None,None
4,1.479424e+09,0.418,0.0,2747.25,None,None,None


In [20]:
throttle['Time'].diff().describe()

count    22117.000000
mean         0.010000
std          0.001973
min          0.001279
25%          0.008471
50%          0.010028
75%          0.011515
max          0.018616
Name: Time, dtype: float64

In [21]:
os.listdir('./Data/ROSBAG/Ch2_002/images/HMB_1/left/')

['frame_1479424215877601572.png',
 'frame_1479424215927376704.png',
 'frame_1479424215977463338.png',
 'frame_1479424216027468320.png',
 'frame_1479424216077857057.png',
 'frame_1479424216127646122.png',
 'frame_1479424216176569089.png',
 'frame_1479424216227025263.png',
 'frame_1479424216276615983.png',
 'frame_1479424216326904145.png',
 'frame_1479424216376863356.png',
 'frame_1479424216426864732.png',
 'frame_1479424216476992614.png',
 'frame_1479424216528110611.png',
 'frame_1479424216577422665.png',
 'frame_1479424216626893541.png',
 'frame_1479424216676870686.png',
 'frame_1479424216727342546.png',
 'frame_1479424216777440999.png',
 'frame_1479424216827011822.png',
 'frame_1479424216877123171.png',
 'frame_1479424216927135266.png',
 'frame_1479424216977107650.png',
 'frame_1479424217027394761.png',
 'frame_1479424217077616319.png',
 'frame_1479424217127387945.png',
 'frame_1479424217177012226.png',
 'frame_1479424217227095540.png',
 'frame_1479424217277050338.png',
 'frame_147942

In [26]:
'frame_1479424215877601572.png'.split('_')[1].split('.')[0]

'1479424215877601572'

In [28]:
left_timestamps = [row.split('_')[1].split('.')[0] for row in os.listdir('./Data/ROSBAG/Ch2_002/images/HMB_1/left/')]

In [40]:
pd.Series([float(row[:10] + '.' + row[10:]) for row in left_timestamps]).diff().describe()

count    4400.000000
mean        0.050009
std         0.000909
min         0.042052
25%         0.049799
50%         0.050005
75%         0.050213
max         0.057304
dtype: float64

In [36]:
float('1479424215877601572'[:10] + '.' + '1479424215877601572'[10:])

1479424215.8776016

### Clean the sterring and throttle data

In [13]:
# Check the image size. Assume each image is the same since it came from the same car.
img = Image.open('./Data/ROSBAG/Ch2_002/images/HMB_1/left/frame_1479424215877601572.png')
width, height = img.size
print(f"Image size: {width}x{height} (Color image)")

Image size: 640x480 (Color image)


In [14]:
def prepare_data_with_labels(df, image_columns, label_columns, target_size=(224, 224)):
    """
    Prepares X (images) and y (labels) for training a CNN.

    Parameters:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        image_columns (list): List of column names with image paths (e.g., ['left_image_path', 'center_image_path', 'right_image_path']).
        label_columns (list): List of column names for labels (e.g., ['steering_angle', 'throttle_cp']).
        target_size (tuple): Target size for resizing images (width, height).

    Returns:
        np.ndarray: Array of image data (X).
        np.ndarray: Array of labels (y).
    """
    X_images = []
    y_labels = []
    
    for _, row in df.iterrows():
        row_images = []
        
        # Process images for the current row
        for col in image_columns:
            image_path = row[col]

            print(f"Looking for image: {image_path}")
            
            if os.path.exists(image_path):  # Ensure file exists
                try:
                    img = Image.open(image_path).resize(target_size)  # Open and resize
                    img_array = np.array(img) / 255.0  # Normalize
                    row_images.append(img_array)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            else:
                print(f"Image not found: {image_path}")
        
        # Check if all images for the row are processed
        if len(row_images) == len(image_columns):
            # Combine images if using multiple cameras
            combined_images = np.stack(row_images, axis=-1)  # Shape: (height, width, 3 * number of cameras)
            X_images.append(combined_images)
            
            # Collect the corresponding labels
            y_labels.append(row[label_columns].values)
            
    df.set_index('Time', inplace = True)
    
    # Convert to NumPy arrays
    X = np.array(X_images, dtype=np.float32)
    y = np.array(y_labels, dtype=np.float32)
    
    return X, y
prepare_data_with_labels(steer,['left_image_path', 'center_image_path', 'right_image_path'],['steering_wheel_angle','steering_wheel_torque','speed'])

Looking for image: None


TypeError: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                   shuffle = False)


In [ ]:
def create_steer_model(input_shape):
    model = Sequential()

    # Input layer
    model.add(Input(shape=input_shape))

    # First convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Second convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Third convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(units=3, activation='linear'))
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    history = model.fit(X_train,y_train,
                       validation_data = (X_test,y_test),
                       epochs = 10,
                       batch_size = 128)

    return model, history, model.evaluate(X_test,y_test)

# Example usage
input_shape = (480, 640, 3)
model = create_steer_model(input_shape)